In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [2]:
from google.colab import drive
from datasets import load_dataset,Audio
drive.mount('/content/drive')

dataset_txt = load_dataset("csv",data_files="/content/drive/MyDrive/texto_final.csv",split="train")

dataset = load_dataset("csv",data_files="/content/drive/MyDrive/datos_finale.csv",split="train")

dataset = dataset.remove_columns(["ID_frase", "conv","speaker","video_name","emotion_x"])
dataset=dataset.rename_column('emocion_code','label')

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from datasets import load_dataset, DatasetDict
import numpy as np
import ast
# Definir una función de procesamiento para convertir las cadenas en arrays de numpy
def convert_to_numpy(example):
    example['audio'] = np.array(ast.literal_eval(example['audio']))
    return example

In [4]:
processed_dataset = dataset.map(convert_to_numpy)

Map:   0%|          | 0/2116 [00:00<?, ? examples/s]

In [7]:
pip install huggingface_hub

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Wav2Vec2ForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np

# Carga los modelos preentrenados
# Modelo de clasificación de texto
text_tokenizer = AutoTokenizer.from_pretrained("Drigoro/roberta_mini")
text_model = AutoModelForSequenceClassification.from_pretrained("Drigoro/roberta_mini")

# Modelo de clasificación de audio
audio_model = Wav2Vec2ForSequenceClassification.from_pretrained("Drigoro/audio_wav2vec")

# Preprocesar los datos
def preprocess_text(batch):
    inputs = text_tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128, return_tensors="pt")
    return inputs

def preprocess_audio(batch):
    # Convertir listas de arrays de audio a tensores PyTorch
    audio_tensors = [torch.tensor(audio, dtype=torch.float32) for audio in batch['audio']]
    # Rellenar los tensores de audio para que tengan la misma longitud
    audio_tensors = nn.utils.rnn.pad_sequence(audio_tensors, batch_first=True)
    return {'input_values': audio_tensors}

def collate_fn(batch):
    texts = [item['text'] for item in batch]
    audios = [item['audio'] for item in batch]
    labels = [item['label'] for item in batch]

    text_inputs = preprocess_text({'text': texts})
    audio_inputs = preprocess_audio({'audio': audios})

    return text_inputs, audio_inputs, torch.tensor(labels)

# Cargar y dividir el conjunto de datos
train_testvalid = processed_dataset.train_test_split(test_size=0.2, seed=1)
train_dataset = train_testvalid['train']
test_dataset = train_testvalid['test']
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

# Definir el modelo ensemble
class EnsembleModel(nn.Module):
    def __init__(self, num_classes):
        super(EnsembleModel, self).__init__()
        self.fc1 = nn.Linear(2 * num_classes, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, text_logits, audio_logits):
        combined = torch.cat((text_logits, audio_logits), dim=1)
        x = torch.relu(self.fc1(combined))
        x = self.fc2(x)
        return x

num_classes = text_model.config.num_labels
ensemble_model = EnsembleModel(num_classes)

# Mover modelos al dispositivo adecuado
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
text_model.to(device)
audio_model.to(device)
ensemble_model.to(device)

# Funciones de predicción
def predict_text(texts):
    texts = {k: v.to(device) for k, v in texts.items()}
    with torch.no_grad():
        outputs = text_model(**texts)
    logits = outputs.logits
    return logits

def predict_audio(audios):
    audios = {"input_values": audios['input_values'].to(device)}
    with torch.no_grad():
        outputs = audio_model(**audios)
    logits = outputs.logits
    return logits

# Entrenar el modelo ensemble
def train_ensemble_model(dataloader, ensemble_model, num_epochs=10, learning_rate=1e-3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ensemble_model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for text_inputs, audio_inputs, labels in dataloader:
            labels = labels.to(device)
            text_logits = predict_text(text_inputs)
            audio_logits = predict_audio(audio_inputs)

            outputs = ensemble_model(text_logits, audio_logits)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}")

    print("Finished Training")

# Evaluar el modelo ensemble
def evaluate_ensemble_model(dataloader, ensemble_model):
    correct = 0
    total = 0

    with torch.no_grad():
        for text_inputs, audio_inputs, labels in dataloader:
            labels = labels.to(device)
            text_logits = predict_text(text_inputs)
            audio_logits = predict_audio(audio_inputs)

            outputs = ensemble_model(text_logits, audio_logits)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the ensemble model on the test set: {accuracy}%')


Epoch [1/10], Loss: 1.1915121973966651
Epoch [2/10], Loss: 1.091493325272821
Epoch [3/10], Loss: 1.0786442842404798
Epoch [4/10], Loss: 1.0673146507931206
Epoch [5/10], Loss: 1.0567220363695666
Epoch [6/10], Loss: 1.048526980264007
Epoch [7/10], Loss: 1.0432968368788935
Epoch [8/10], Loss: 1.0442162887105402
Epoch [9/10], Loss: 1.041189421390025
Epoch [10/10], Loss: 1.0385521592677764
Finished Training
Accuracy of the ensemble model on the test set: 62.971698113207545%


In [ ]:
# Entrenar el modelo
train_ensemble_model(train_dataloader, ensemble_model)

In [8]:
# Evaluar el modelo
evaluate_ensemble_model(test_dataloader, ensemble_model)

Accuracy of the ensemble model on the test set: 62.971698113207545%
